# Register an inference spark environment
Custom [image](https://github.com/Azure/mmlspark/blob/master/tools/docker/minimal/Dockerfile) with Spark 3.0.1 from mmlspark.

## Get the Workspace

In [ ]:
import azureml
from azureml.core import Workspace
from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.core.environment import Environment

workspace_name = '<YOUR-WORKSPACE-NAME>'
resource_group = '<YOUR-RESOURCE-GROUP>'
subscription_id = '<YOUR-SUBSCRIPTION-ID>'

def get_workspace(workspace_name, resource_group, subscription_id):
  svc_pr = ServicePrincipalAuthentication(
      tenant_id = dbutils.secrets.get(scope = "azure-key-vault", key = "tenant-id"),
      service_principal_id = dbutils.secrets.get(scope = "azure-key-vault", key = "cliente-id-custom-role"),
      service_principal_password = dbutils.secrets.get(scope = "azure-key-vault", key = "cliente-secret-custom-role"))

  workspace = Workspace.get(name = workspace_name,
                            resource_group = resource_group,
                            subscription_id = subscription_id,
                            auth=svc_pr)
  
  return workspace

workspace = get_workspace(workspace_name, resource_group, subscription_id)

## Create the environment

In [ ]:
from azureml.core.environment import Environment
from azureml.core.webservice import LocalWebservice
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig, Model

# BASE IMAGE from https://github.com/Azure/mmlspark/blob/master/tools/docker/minimal/Dockerfile 
dockerfile = """
FROM mcr.microsoft.com/azureml/minimal-ubuntu18.04-py37-cpu-inference:latest

USER root:root

ARG SPARK_VERSION=3.2.0
RUN mkdir -p /usr/share/man/man1 && \
    apt-get -qq update && apt-get \
    -qq upgrade && apt-get -qq -y install curl bzip2 && \
    curl -sSL https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -o /tmp/miniconda.sh && \
    bash /tmp/miniconda.sh -bfp /usr/local && rm -rf /tmp/miniconda.sh && \
    conda install -y python=3 && \
    conda update conda && \
    apt-get install openjdk-8-jre -y && \
    conda install -c conda-forge pyspark=${SPARK_VERSION} && \
    apt-get -qq -y remove curl bzip2 && \
    apt-get -qq -y autoremove && \
    apt-get autoclean && \
    rm -rf /var/lib/apt/lists/* /var/log/dpkg.log && \
    conda clean --all --yes

RUN pip install mlflow pyspark pandas azureml-defaults

ENV PATH /opt/conda/bin:$PATH
ENV JAVA_HOME /usr/lib/jvm/java-1.8.0-openjdk-amd64
"""

# Create an environment to be able to customize our dependencies
my_spark_env = Environment('spark-env-custom')

# Now we can indicate we will use our custom Base Image
my_spark_env.docker.base_image = None
my_spark_env.docker.base_dockerfile = dockerfile
my_spark_env.python.user_managed_dependencies = True

# It's very important to use this parameter
my_spark_env.inferencing_stack_version='latest'

## Register the environment

In [ ]:
my_spark_env.register(workspace)